In [1]:
set.seed(1919)
library(dplyr)
library(ggplot2)
library(randomForest)
library(ROCR)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:ggplot2’:

    margin

The following object is masked from ‘package:dplyr’:

    combine

Loading required package: gplots

Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess



In [3]:
dat <- read.table("RFE_data.txt", header = T, sep = "\t")
dat <- as.data.frame(scale(dat))
sample_dat <- read.table("ZTX/scirep_classes.txt", header = T, sep = ",", stringsAsFactors = F)
sample_class <- as.factor(arrange(sample_dat, sample_id)$label)

In [5]:
## Find Best mtry in RF
for(i in 1:8){
    rf_classifier = randomForest(x = dat, y = sample_class,  ntree = 400, mtry = i)
    print(mean(rf_classifier$err.rate))
}

[1] 0.4487145
[1] 0.462312
[1] 0.4469695
[1] NaN
[1] 0.4519014
[1] NaN
[1] 0.4451666
[1] 0.438807


In [6]:
## RF
rf_classifier = randomForest(x = RF_train_sample, y = RF_train_sample_class,  ntree = 400, mtry = 3)
rf_classifier
pre_prob <- predict(rf_classifier, dat, type = "prob")

ERROR: Error in randomForest(x = RF_train_sample, y = RF_train_sample_class, : object 'RF_train_sample' not found


In [26]:
pre_prob

,Colorectal Cancer,Healthy Control,Pancreatic Cancer,Prostate Cancer
Sample_1S1,0.9700,0.0250,0.0050,0.0000
Sample_1S10,0.4050,0.1450,0.0125,0.4375
Sample_1S11,0.6650,0.3200,0.0075,0.0075
Sample_1S12,0.8000,0.0575,0.0225,0.1200
Sample_1S13,0.5625,0.3350,0.0725,0.0300
Sample_1S14,0.9250,0.0625,0.0100,0.0025
Sample_1S15,0.7350,0.2575,0.0000,0.0075
Sample_1S16,0.9000,0.0475,0.0375,0.0150
Sample_1S17,0.6025,0.3075,0.0825,0.0075
Sample_1S18,0.9525,0.0375,0.0025,0.0075


In [30]:
## Colorectal Cancer的ROC
positive_class <- 'Colorectal Cancer'
test_labels <- vector('integer', nrow(pre_prob))
test_labels[sample_class != positive_class] <- 0
test_labels[sample_class == positive_class] <- 1
pred <- prediction(pre_prob[, positive_class], test_labels)
roc <- performance(pred, 'tpr', 'fpr') 
png("Col_Can_ROC_RF.png")
plot(roc, main = 'ROC Curve for Colorectal Cancer', sub = 'AUC = 0.998')
dev.off()
auc <- performance(pred, 'auc')
cat('auc =', auc@y.values[[1]], '\n')

png 
  2

auc = 0.9978041 


In [33]:
## Healthy Control的ROC
positive_class <- 'Healthy Control'
test_labels <- vector('integer', nrow(pre_prob))
test_labels[sample_class != positive_class] <- 0
test_labels[sample_class == positive_class] <- 1
pred <- prediction(pre_prob[, positive_class], test_labels)
roc <- performance(pred, 'tpr', 'fpr') 
png("Healthy_Control_ROC_RF.png")
plot(roc, main = 'ROC Curve for Healthy Control', sub = 'AUC = 0.991')
dev.off()
auc <- performance(pred, 'auc')
cat('auc =', auc@y.values[[1]], '\n')

png 
  2

auc = 0.9908511 


In [35]:
## Pancreatic Cancer的ROC
positive_class <- 'Pancreatic Cancer'
test_labels <- vector('integer', nrow(pre_prob))
test_labels[sample_class != positive_class] <- 0
test_labels[sample_class == positive_class] <- 1
pred <- prediction(pre_prob[, positive_class], test_labels)
roc <- performance(pred, 'tpr', 'fpr') 
png("Pan_Can_ROC_RF.png")
plot(roc, main = 'ROC Curve for Pancreatic Cancer', sub = 'AUC = 0.931')
dev.off()
auc <- performance(pred, 'auc')
cat('auc =', auc@y.values[[1]], '\n')

png 
  2

auc = 0.9306306 


In [37]:
## Prostate Cancer的ROC
positive_class <- 'Prostate Cancer'
test_labels <- vector('integer', nrow(pre_prob))
test_labels[sample_class != positive_class] <- 0
test_labels[sample_class == positive_class] <- 1
pred <- prediction(pre_prob[, positive_class], test_labels)
roc <- performance(pred, 'tpr', 'fpr') 
png("Pro_Can_ROC_RF.png")
plot(roc, main = 'ROC Curve for Prostate Cancer', sub = 'AUC = 0.999')
dev.off()
auc <- performance(pred, 'auc')
cat('auc =', auc@y.values[[1]], '\n')

png 
  2

auc = 0.9991039 
